In [1]:
import pandas as pd
import numpy as np
import datetime
from scorepi import *
from epiweeks import Week
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pickle
from pathlib import Path
import requests
import os

import warnings
warnings.filterwarnings('ignore')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
!python --version

Python 3.10.9


In [3]:
pd.__version__

'1.4.4'

Python and pandas versions must be enforced for this code to work. There is a copying error which occurs in the scorepi class constructors when using newer versions.

# Load Data

At least for now, I'm using Clara's functions for pulling predictions and surveillance data. We will probably want to set up the Flusight repo as a submodule and pull new data on a schedule. 


Here's how it's currently implemented:
1. For each selected model and date, we save each forecast submission as-is in parquet format.
2. For each selected model and date, we read the corresponding parquets and concatenate into a single data frame.
3. We instantiate the Forecast_Eval class with a given start and end week, and call the format_forecasts_all method on the concatenated predictions dataframe, which filters for quantile predictions, removes dates after the end week, enforces datatypes, and returns the formatted dataframe to be used in scoring. (NOTE: start week is not used, what's up with that?)

Automating this workflow will probably involve:
1. Calculate scorings for all historical data once and save.
2. Set up the Flusight repo as a submodule and pull new data on a schedule.
3. It might be possible to keep the implementation with parquets if github can run it (pulling from flusight repo and writing and reading parquets within a python script). This would eliminate (2.) and could avoid problems when making epistorm-evaluations a submodule of epistorm-dashboard, since it also has the flusight repo as a submodule.
4. Every time new data is pulled, calculate scoring for only the new data and append to existing scoring files. Do we need to re-calculate scores for previous weeks in case surveillance data changes retrospectively?
5. We might generate separate files with transformed data for charts in the dashboard, if so either:
    1. Do that here, add epistorm-evaluations as a submodule of epistorm-dashboard, and pull the transformed scores on a schedule.
    2. Add epistorm-evaluations as a submodule of epistorm-dashboard, pull the scoring files on a schedule, and trigger a script within epistorm-dashboard to create the transformed scoring files.


In [2]:
# functions to download flusight model predictions and surveillance data

def pull_flusight_predictions(model,date):
    """pull_flusight_predictions. Load predictions of the model saved by the Flusight Forecast hub

    Parameters
    ----------
    model : str
        Model name on the
    dates : list or string
        List of potential dates in the iso format, e.g., 'yyyy-mm-dd', for the submission.
    """
    predictions = None
    
    url = f"https://raw.githubusercontent.com/cdcepi/Flusight-forecast-hub/main/model-output/{model}/{date}-{model}"
    for ext in [".csv",".gz",".zip",".csv.zip",".csv.gz"]:
        try:
            predictions = pd.read_csv(url+ext,dtype={'location':str},parse_dates=['target_end_date'])
        except:
            pass
    if predictions is None:
        print(f"Data for model {model} and date {date} unavailable")
    return predictions


def pull_surveillance_data():
    """pull_surveillance_data. Load hospitalization admissions surveillance data
    """
    
    url = f"https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/main/target-data/target-hospital-admissions.csv"
    return pd.read_csv(url, dtype={'location':str})


In [3]:
# download and save surveillance data
surv = pull_surveillance_data() 
surv = surv[surv.date >= '2023-06-01'] # filtered dates after june 2023 since we are only looking at 2023-24 season
surv.to_parquet(f"./dat/target-hospital-admissions.pq", index=False)

Clara was hard-coding dates to pull each time she manually ran this notebook. Since we want scores for all historical data, I'm using all dates from the surveillance file. I don't think this should cause a bug or miss any data but it would be good to have a second opinion.

Once we've got all the historical scores, if we're only scoring new predictions every week, should we only use the most recent surveillance date?

In [4]:
#selecting all target dates that exist in the surveillance file
dates = pd.unique(surv.date)

#selecting just models used in the dashboard for now
#will need to expand eventually whether we keep the parquet implementation or pull files from the flusight repo as a submodule
models = ['CEPH-Rtrend_fluH', 'FluSight-baseline', 'FluSight-ensemble', 'MIGHTE-Nsemble', 'MOBS-GLEAM_FLUH', 'NU_UCSD-GLEAM_AI_FLUH']

In [5]:
for model in models:
    for date in dates:
        try:
            predictions = pull_flusight_predictions(model,date)

            predictions.to_parquet(f'./dat/{model}_{date}.pq', index=False)
        except Exception as e:
            print(e)

Data for model CEPH-Rtrend_fluH and date 2023-10-07 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-09-30 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-09-23 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-09-16 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-09-09 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-09-02 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-08-26 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-08-19 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model CEPH-Rtrend_fluH and date 2023-08-12 unavailable
'NoneType' object has no attribute 'to_p

Data for model MIGHTE-Nsemble and date 2023-06-10 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MIGHTE-Nsemble and date 2023-06-03 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MOBS-GLEAM_FLUH and date 2023-10-07 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MOBS-GLEAM_FLUH and date 2023-09-30 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MOBS-GLEAM_FLUH and date 2023-09-23 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MOBS-GLEAM_FLUH and date 2023-09-16 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MOBS-GLEAM_FLUH and date 2023-09-09 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MOBS-GLEAM_FLUH and date 2023-09-02 unavailable
'NoneType' object has no attribute 'to_parquet'
Data for model MOBS-GLEAM_FLUH and date 2023-08-26 unavailable
'NoneType' object has no attribute 'to_parquet'
Dat

# Classes and Methods

In [37]:
# functions for calculating scores of the forecasts against the surveillance data

class Forecast_Eval:
    """ Used for scoring and evaluating flu forecasting predictions from the Flusight challenge for the 
        2023-24 season
    """
    
    def __init__(self, df, obsdf, target,  start_week = False, end_week = False):
        self.df = df # pandas Dataframe: input dataframe of forecasts with all scenarios, locations, and quantiles
        self.obsdf = obsdf # pandas Dataframe: input of surveillance data of interest
        self.target = target # str: target metric of interest (case, death, hospitalization)
        self.start_week = start_week # epiweek: beginning of observations of interest
        self.end_week = end_week # epiweek: end of observations of interest
        
          
    def get_observations(self, target_location):
        """ get_observations. Load and filter surveillance data for a certain location.
        Parameters
        ----------
        target_location : str
            location to filter surveillance data by
        """
        
        if self.target == 'hosp':
            target_obs = 'hospitalization'
        else:
            target_obs = self.target
            
        # read in observations dataframe
        observations = self.obsdf.copy().drop(columns= ['Unnamed: 0', 'weekly_rate'])
        observations['date'] = pd.to_datetime(observations['date'])

        #filter start - end week
        if self.start_week:
            observations = observations[(observations['date'] >= pd.to_datetime(self.start_week.startdate())) ]
            
        if self.end_week:
            observations = observations[(observations['date'] <= pd.to_datetime(self.end_week.enddate()))]
                                
        #filter location
        observations = observations[observations['location'] == target_location]

        #aggregate to weekly
        observations = observations.groupby(['location', pd.Grouper(key='date', freq='W-SAT')]).sum().reset_index()

        #transform to Observation object
        observations = Observations(observations)

        return observations
    
    
    def format_forecasts_all(self, dfformat):
        """ format_forecasts_all. Get forecasts into standard format to use for scoring.
        Parameters
        ----------
        dfformat : pandas DataFrame
            dataframe of forecast output used for formatting
        """
        
        pred = dfformat.copy()
        pred = pred[pred.output_type == 'quantile'] # only keep quantile predictions
        pred['target_end_date'] = pd.to_datetime(pred['target_end_date']) #make sure dates are in datetime format
        if self.start_week:
            pred = pred[(pred['target_end_date'] >= pd.to_datetime(self.start_week.startdate()))] # filter dates
        
        if self.end_week:
            pred = pred[(pred['target_end_date'] <= pd.to_datetime(self.end_week.enddate()))] # filter dates
        
        pred['output_type_id'] = pred["output_type_id"].astype(float) # make sure quantile levels are floats
        
        return pred
        
    

class Scoring(Forecast_Eval):
    """ calculate score values for probabilistic epidemic forecasts 
    find WIS, MAPE, and coverage over whole projection window as well as timestamped for every week.
    uses scorepi package to calculate the scores  (https://github.com/gstonge/scorepi/tree/main)
    score dataframe must have 'Model' column to differentiate and calculate scores for different models
    """
    
    def __init__(self, df, obsdf, target, start_week = False, 
                 end_week = False):
        super().__init__(df, obsdf, target, start_week, end_week)
        
    def get_all_average_scores(self, models):
        """ get_all_average_scores. Calculate all score in scorepi package that average over the full forecast
        time series. 
        
        Parameters
        ----------
        models: list
            list of models that the scores will be calculated for, each element is a string corresponding to
            a forecast model's name from the Model column of the forecast dataframe
        """
        
        pred1 = self.df.copy() # dataframe that will be scored
        loclist = list(pred1.location.unique()) 
        
        
        allscore = {}
        for model in models:
            allscore[model] = {}
            for target_location in loclist:
                if target_location == '72':
                    continue
                #print(target_location)
                
                observations = self.get_observations(target_location) # get surveillance data for target location 

                # filter by model and location
                pred = pred1[(pred1.Model==model) & (pred1['location']==target_location) ] 
                # make into Predictions object
                pred = Predictions(pred, t_col = 'target_end_date', quantile_col = 'output_type_id')
                observations = Observations(observations[observations.date<=pred.target_end_date.max()])
                #calculate scores
                d,_ = score_utils.all_scores_from_df(observations, pred, mismatched_allowed=False) 

                # save in dictionary
                allscore[model][target_location] = d
            
        
        return allscore
    
    def organize_average_scores(self, want_scores, models):
        """ organize_average_scores. save average scores of interest into a pandas dataframe
        
        Parameters
        ----------
        want_scores: list
            list of scores you want to save in the dataframe
            wis is 'wis_mean', and all coverages are '10_cov', '20_cov', ... '95_cov' etc.
        models: list
            list of models that the scores will be calculated for, each element is a string correspongding to
            a forecast model's name from the Model column of the forecast dataframe. 
            used for get_all_average_scores function call.
        """
        
        average_scores = pd.DataFrame()
        
        allscore = self.get_all_average_scores(models) #calculate all average scores
        
        for model in allscore.keys():
            scoresmod = allscore[model]
            for loc in scoresmod.keys():
                    
                scoresloc = scoresmod[loc]

                scoredict = {'Model': model ,'location': loc}
                for score in want_scores: # only save scores input into want_scores
                    scoredict[score] = scoresloc[score]

                average_scores = pd.concat([average_scores, pd.DataFrame(scoredict, index=[0])])

        average_scores = average_scores.reset_index() 
        average_scores = average_scores.drop(columns=['index'])
        
        return average_scores
    
    def get_all_timestamped_scores(self, models):
        """ get_all_timestamped_scores. Calculate all scores in scorepi package for each week of the full forecast
        time series. 
        
        Parameters
        ----------
        models: list
            list of models that the scores will be calculated for, each element is a string corresponding to
            a forecast model's name from the Model column of the forecast dataframe
        """
        
        pred = self.df.copy() # dataframe used for scoring
        loclist = list(pred.location.unique())
        
        allscore = {}
        
        for model in models:
            allscore[model] = {}
            for target_location in loclist:
                    
                observations = self.get_observations(target_location) # get surveillance data for target location
                
                try:
                    predss = pred[pred['location'] == target_location] #filter by location
                    # format forecasts into Predictions scorepi objec
                    predss = Predictions(predss, t_col = 'target_end_date', quantile_col = 'output_type_id')
                    
                    if len(predss)==0:
                        continue
                    
                    allscore[model][target_location] = {}
                    # loop over all time points in the predictions
                    for t in predss.target_end_date.unique():
                        prednew = predss[predss.target_end_date == t]
                        obsnew = observations[observations.date == t]

                        obsnew = Observations(obsnew)
                        prednew = Predictions(prednew, t_col = 'target_end_date', quantile_col = 'output_type_id')

                        # calculate scores
                        d = score_utils.all_timestamped_scores_from_df(obsnew, prednew)

                        allscore[model][target_location][t] = d
                except Exception as e:
                    print(e)
        
        return allscore
    
    
    def organize_timestamped_scores(self, want_scores, models):
        """ organize_timestamped_scores. save timestamped scores of interest into a pandas dataframe
        
        Parameters
        ----------
        want_scores: list
            list of scores you want to save in the dataframe
            wis is 'wis'
        models: list
            list of models that the scores will be calculated for, each element is a string correspongding to
            a forecast model's name from the Model column of the forecast dataframe. 
            used for get_all_timestamped_scores function call.
        """
    
        
        time_scores = pd.DataFrame()
        
        # calculate all scores evaluated for each time point
        allscore = self.get_all_timestamped_scores(models=models)
        
        for model in allscore.keys():
            scoremod = allscore[model]
        
            for loc in scoremod.keys():
                    
                scoresloc = scoremod[loc]

                for t in scoresloc.keys():
                    tdf = scoresloc[t]

                    scoredict = {'Model':model ,'location':loc, 'target_end_date':t}
                    for score in want_scores:
                        scoredict[score] = tdf[score]

                    # save scores in want_scores into a dataframe
                    time_scores = pd.concat([time_scores, pd.DataFrame(scoredict, index=[0])])

        
        time_scores = time_scores.reset_index() 
        time_scores = time_scores.drop(columns=['index'])
        
        return time_scores
    
    
    def get_mape(self):
        """ get_mape. Calculate MAPE (mean absolute percentage error) for each date of a forecast. If 
            surveillance data point is equal to zero, the score is undefined (Nan).
        
        """
        
        predictions = self.df.copy()
        
        # get point forecast, here we say it is the median
        predictions = predictions[predictions['output_type_id'] == 0.5] # get point forecast, here we say it is the median

        mapedf = pd.DataFrame()

        # find mape for a given model and location over projection period
        for model in predictions.Model.unique():
            for target_location in predictions.location.unique():

                    if target_location in ['60','66','69', '72', '78']:
                        continue

                    observations = self.get_observations(target_location)
                    

                    pred = predictions[(predictions.location == target_location) & (predictions.Model==model)]
                    pred = Predictions(pred, t_col = 'target_end_date',quantile_col='output_type_id')
                    
                    observations = observations[observations.date.isin(pred.target_end_date.unique())]

                    n = observations.shape[0]

                    realvals = list(observations.value)
                    predvals = list(pred.value)

                    
                    if len(predvals) == 0:
                        continue

                    if realvals[0] == 0:
                        n = n - 1
                        continue

                    err = abs((realvals[0]-predvals[0])/realvals[0]) # find relative error


                    if n == 0:
                        mape = None
                    else:
                        mape = err # calculate mape


                    data = {'Model': model,'Location': target_location, 'MAPE':mape}

                    # store in pandas DataFrame
                    newdf = pd.DataFrame(data, index=[1])

                    mapedf = pd.concat([mapedf, newdf])

        mapedf = mapedf.reset_index()
        mapedf = mapedf.drop(['index'], axis=1)

        return mapedf
 

# Calculate Scores

## Instantiate Forecast_Eval Class and Format Data for Scoring

I'm keeping the dates and models specified earlier.

In [7]:
# put all forecasts into one dataframe
predictionsall = pd.DataFrame()
for model in models:
    for date in dates:
        try:
            predictions = pd.read_parquet(f'./dat/{model}_{date}.pq')
            predictions['Model'] = model
            predictionsall = pd.concat([predictionsall, predictions])
        except Exception as e:
            print(e)

[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-10-07.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-09-30.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-09-23.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-09-16.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-09-09.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-09-02.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-08-26.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-08-19.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-08-12.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-08-05.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-07-29.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-07-22.pq'
[Errno 2] No such file or directory: './dat/CEPH-Rtrend_fluH_2023-07-15.pq'
[Errno 2] No

In [38]:
# format forecasts in order to calculate scores
# input start and end weeks for the period of interest
test = Forecast_Eval(df=pd.DataFrame(), obsdf=surv, target='hosp', 
                            start_week = Week(2023,40), end_week = Week(2024, 17))
predsall = test.format_forecasts_all( dfformat = predictionsall)

In [39]:
predsall

,reference_date,target,horizon,target_end_date,location,output_type,output_type_id,value,Model
0,2024-04-27,wk inc flu hosp,0,2024-04-27,01,quantile,0.010,3.000,CEPH-Rtrend_fluH
4,2024-04-27,wk inc flu hosp,0,2024-04-27,01,quantile,0.025,5.000,CEPH-Rtrend_fluH
8,2024-04-27,wk inc flu hosp,0,2024-04-27,01,quantile,0.050,7.000,CEPH-Rtrend_fluH
12,2024-04-27,wk inc flu hosp,0,2024-04-27,01,quantile,0.100,9.000,CEPH-Rtrend_fluH
16,2024-04-27,wk inc flu hosp,0,2024-04-27,01,quantile,0.150,10.000,CEPH-Rtrend_fluH
...,...,...,...,...,...,...,...,...,...
5975,2023-12-02,wk inc flu hosp,3,2023-12-23,US,quantile,0.850,20509.175,NU_UCSD-GLEAM_AI_FLUH
5976,2023-12-02,wk inc flu hosp,3,2023-12-23,US,quantile,0.900,24627.713,NU_UCSD-GLEAM_AI_FLUH
5977,2023-12-02,wk inc flu hosp,3,2023-12-23,US,quantile,0.950,32274.821,NU_UCSD-GLEAM_AI_FLUH
5978,2023-12-02,wk inc flu hosp,3,2023-12-23,US,quantile,0.975,41471.763,NU_UCSD-GLEAM_AI_FLUH


## WIS

In [40]:
# calculate wis for all forecasts

dfwis = pd.DataFrame()
for horizon in [0, 1, 2,3]:
    for model in models:
        for date in dates: 
            start_week = Week.fromdate(pd.to_datetime(date)) # week of submission date
            end_week = start_week + 3 # target end date of last horizon
            
            # filter by horizon, model and submission date
            pred = predsall[(predsall.horizon==horizon) & (predsall.Model == model) & \
                            (predsall.reference_date == date)]
            if len(pred)==0:
                continue
            
            # calculate wis for each week
            test = Scoring(df=pred, obsdf=surv, target='hosp',
                            start_week = start_week, end_week = end_week)

            out = test.organize_timestamped_scores(want_scores = ['wis'], models = [model])
            
            out['horizon'] = horizon
            out['reference_date'] = date
            
            dfwis = pd.concat([dfwis, out])

In [12]:
dfwis.to_csv('./WIS.csv', index=False)

In [41]:
dfwis

,Model,location,target_end_date,wis,horizon,reference_date
0,CEPH-Rtrend_fluH,01,2024-04-27,1.912889,0,2024-04-27
1,CEPH-Rtrend_fluH,02,2024-04-27,3.405661,0,2024-04-27
2,CEPH-Rtrend_fluH,04,2024-04-27,23.574783,0,2024-04-27
3,CEPH-Rtrend_fluH,05,2024-04-27,3.179565,0,2024-04-27
4,CEPH-Rtrend_fluH,06,2024-04-27,6.374348,0,2024-04-27
...,...,...,...,...,...,...
47,NU_UCSD-GLEAM_AI_FLUH,53,2023-12-23,79.365652,3,2023-12-02
48,NU_UCSD-GLEAM_AI_FLUH,54,2023-12-23,53.268863,3,2023-12-02
49,NU_UCSD-GLEAM_AI_FLUH,55,2023-12-23,113.721773,3,2023-12-02
50,NU_UCSD-GLEAM_AI_FLUH,56,2023-12-23,37.325938,3,2023-12-02


## WIS Ratio

In [43]:
# compute wis ratio, comparing the Flusight models' forecast scores to the Flusight baseline model
# divide flusight models by flusight baseline WIS scores at each location, week, horizon, location
dfwis = pd.read_csv('./WIS.csv')
baseline = dfwis[dfwis.Model == 'FluSight-baseline'] 
baseline = baseline.rename(columns={'wis':'wis_baseline', 'Model':'baseline'})
dfwis_test = dfwis[dfwis.Model != 'FluSight-baseline']

dfwis_ratio = pd.merge(dfwis_test, baseline, how='inner', on = ['location', 'target_end_date',
                                                                'horizon', 'reference_date'])

# calculate wis ratio
dfwis_ratio['wis_ratio'] = dfwis_ratio['wis']/dfwis_ratio['wis_baseline']

In [16]:
dfwis_ratio.to_csv('./WIS_ratio.csv', index=False)

In [44]:
dfwis_ratio

,Model,location,target_end_date,wis,horizon,reference_date,baseline,wis_baseline,wis_ratio
0,CEPH-Rtrend_fluH,01,2024-04-27,1.912889,0,2024-04-27,FluSight-baseline,3.186017,0.600401
1,FluSight-ensemble,01,2024-04-27,2.005467,0,2024-04-27,FluSight-baseline,3.186017,0.629459
2,MIGHTE-Nsemble,01,2024-04-27,2.774675,0,2024-04-27,FluSight-baseline,3.186017,0.870891
3,MOBS-GLEAM_FLUH,01,2024-04-27,9.271692,0,2024-04-27,FluSight-baseline,3.186017,2.910120
4,NU_UCSD-GLEAM_AI_FLUH,01,2024-04-27,6.820537,0,2024-04-27,FluSight-baseline,3.186017,2.140772
...,...,...,...,...,...,...,...,...,...
27397,MIGHTE-Nsemble,72,2023-11-04,29.624689,3,2023-10-14,FluSight-baseline,24.647738,1.201923
27398,CEPH-Rtrend_fluH,US,2023-11-04,437.658750,3,2023-10-14,FluSight-baseline,551.338550,0.793811
27399,FluSight-ensemble,US,2023-11-04,275.942167,3,2023-10-14,FluSight-baseline,551.338550,0.500495
27400,MIGHTE-Nsemble,US,2023-11-04,230.569058,3,2023-10-14,FluSight-baseline,551.338550,0.418199


## Coverage

In [45]:
dfcoverage = pd.DataFrame()

for date in dates:
    for model in models:
         
        start_week = Week.fromdate(pd.to_datetime(date)) # week of submission date
        end_week = start_week + 3 # target end date of last horizon

        # filter by model and submission date, only look at horizon 0-3
        pred = predsall[(predsall.Model == model) & \
                        (predsall.reference_date == date) & (predsall.horizon >=0)]
        if len(pred)==0:
            continue

        # calculate wis for each week
        test = Scoring(df=pred, obsdf=surv, target='hosp',  
                        start_week = start_week, end_week = end_week)

        out = test.organize_average_scores(want_scores=['10_cov', '20_cov', '30_cov', '40_cov', '50_cov',
            '60_cov', '70_cov', '80_cov', '90_cov', '95_cov', '98_cov'], models = [model])

        out['horizon'] = horizon
        out['reference_date'] = date

        dfcoverage = pd.concat([dfcoverage, out])

In [19]:
dfcoverage.to_csv('./coverage.csv', index=False)

In [46]:
dfcoverage

,Model,location,10_cov,20_cov,30_cov,40_cov,50_cov,60_cov,70_cov,80_cov,90_cov,95_cov,98_cov,horizon,reference_date
0,CEPH-Rtrend_fluH,01,0.0,1.00,1.00,1.00,1.0,1.0,1.00,1.00,1.00,1.00,1.00,3,2024-04-27
1,CEPH-Rtrend_fluH,02,0.0,0.00,0.00,0.00,0.0,0.0,0.00,1.00,1.00,1.00,1.00,3,2024-04-27
2,CEPH-Rtrend_fluH,04,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,1.00,1.00,3,2024-04-27
3,CEPH-Rtrend_fluH,05,0.0,0.00,0.00,1.00,1.0,1.0,1.00,1.00,1.00,1.00,1.00,3,2024-04-27
4,CEPH-Rtrend_fluH,06,1.0,1.00,1.00,1.00,1.0,1.0,1.00,1.00,1.00,1.00,1.00,3,2024-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,MOBS-GLEAM_FLUH,53,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.25,0.25,0.50,3,2023-10-14
48,MOBS-GLEAM_FLUH,54,0.0,0.25,0.25,0.25,0.5,0.5,0.50,1.00,1.00,1.00,1.00,3,2023-10-14
49,MOBS-GLEAM_FLUH,55,0.0,0.00,0.00,0.00,0.0,0.0,0.25,0.25,0.25,0.25,0.75,3,2023-10-14
50,MOBS-GLEAM_FLUH,56,0.0,0.00,0.00,0.25,0.5,0.5,0.50,0.50,0.75,0.75,0.75,3,2023-10-14


# MAPE

In [47]:
# calculate MAPE for all forecasts

dfmape = pd.DataFrame()
#dic = {}
for horizon in [0, 1, 2,3]:
    for model in models:
        for date in dates: 
            start_week = Week.fromdate(pd.to_datetime(date)) # week of submission date
            end_week = start_week + 3 # target end date of last horizon
            
            # filter by horizon, model and submission date
            pred = predsall[(predsall.horizon==horizon) & (predsall.Model == model) & \
                            (predsall.reference_date == date)]
            if len(pred)==0:
                continue
            
            # calculate mape for each week
            test = Scoring(df=pred, obsdf=surv, target='hosp',
                            start_week = start_week, end_week = end_week)

            out = test.get_mape()
            
            out['horizon'] = horizon
            out['reference_date'] = date
            
            dfmape = pd.concat([dfmape, out])

In [48]:
dfmape.to_csv('./MAPE.csv', index=False)

In [50]:
dfmape

,Model,Location,MAPE,horizon,reference_date
0,CEPH-Rtrend_fluH,01,0.125000,0,2024-04-27
1,CEPH-Rtrend_fluH,02,2.000000,0,2024-04-27
2,CEPH-Rtrend_fluH,04,0.367925,0,2024-04-27
3,CEPH-Rtrend_fluH,05,0.200000,0,2024-04-27
4,CEPH-Rtrend_fluH,06,0.013245,0,2024-04-27
...,...,...,...,...,...
47,NU_UCSD-GLEAM_AI_FLUH,53,0.700116,3,2023-12-02
48,NU_UCSD-GLEAM_AI_FLUH,54,0.821854,3,2023-12-02
49,NU_UCSD-GLEAM_AI_FLUH,55,0.768278,3,2023-12-02
50,NU_UCSD-GLEAM_AI_FLUH,56,2.546345,3,2023-12-02
